In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telegram-spam-or-ham/dataset.csv


# A Word-Level Neural Language Model And Generate Text 

In [2]:
# load text
filename = '/kaggle/input/telegram-spam-or-ham/dataset.csv'
file = open(filename, 'rt')
text = file.read()
file.close()

In [3]:
import pandas as pd

def load_doc_from_csv(file_path, column_name):
    """
    Load document content from a CSV file.

    Args:
    - file_path (str): The path to the CSV file.
    - column_name (str): The name of the column containing the document content.

    Returns:
    - str: Document content.
    """
    try:
        # Load CSV file
        df = pd.read_csv(file_path)
        # Extract document content from specified column
        doc = df['text'].iloc[0]  # Assuming there's only one document in the CSV file
        return doc
    except FileNotFoundError:
        print("File not found:", file_path)
        return None

# Specify the file path and column name
file_path = '/kaggle/input/telegram-spam-or-ham/dataset.csv'  # Replace 'your_csv_file.csv' with the actual file path
column_name = 'text'  # Replace 'column_containing_document_content' with the actual column name

# Load document
doc = load_doc_from_csv(file_path, column_name)
if doc is not None:
    print(doc[:200])  # Print the first 200 characters of the document

naturally irresistible your corporate identity lt is really hard to recollect a company the market is full of suqgestions and the information isoverwhelminq but a good catchy logo stylish statlonery a


In [4]:
import string

# turn a doc into clean tokens
def clean_doc(doc):
    """
    Clean and tokenize a document.

    Args:
    - doc (str): The document to be cleaned.

    Returns:
    - list: List of clean tokens.
    """
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation and make lower case
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table).lower() for word in tokens if word.isalpha()]
    return tokens

In [5]:
# Clean document
tokens = clean_doc(doc)

# Print the first 200 tokens
print("First 200 tokens:")
print(tokens[:200])

# Print total number of tokens and unique tokens
total_tokens = len(tokens)
unique_tokens = len(set(tokens))
print('Total Tokens:', total_tokens)
print('Unique Tokens:', unique_tokens)

First 200 tokens:
['naturally', 'irresistible', 'your', 'corporate', 'identity', 'lt', 'is', 'really', 'hard', 'to', 'recollect', 'a', 'company', 'the', 'market', 'is', 'full', 'of', 'suqgestions', 'and', 'the', 'information', 'isoverwhelminq', 'but', 'a', 'good', 'catchy', 'logo', 'stylish', 'statlonery', 'and', 'outstanding', 'website', 'will', 'make', 'the', 'task', 'much', 'easier', 'we', 'do', 'not', 'promise', 'that', 'havinq', 'ordered', 'a', 'iogo', 'your', 'company', 'will', 'automaticaily', 'become', 'a', 'world', 'ieader', 'it', 'isguite', 'ciear', 'that', 'without', 'good', 'products', 'effective', 'business', 'organization', 'and', 'practicable', 'aim', 'it', 'will', 'be', 'hotat', 'nowadays', 'market', 'but', 'we', 'do', 'promise', 'that', 'your', 'marketing', 'efforts', 'will', 'become', 'much', 'more', 'effective', 'here', 'is', 'the', 'list', 'of', 'clear', 'benefits', 'creativeness', 'hand', 'made', 'original', 'logos', 'specially', 'done', 'to', 'reflect', 'your', 'd

In [6]:
# Define the length of each sequence
sequence_length = 50 + 1

# Initialize a list to store sequences
sequences = []

# Generate sequences of tokens
for i in range(sequence_length, len(tokens)):
    # Select sequence of tokens
    sequence = tokens[i - sequence_length: i]
    # Convert the sequence into a line
    line = ' '.join(sequence)
    # Store the line
    sequences.append(line)

# Print the total number of sequences
print('Total Sequences:', len(sequences))

Total Sequences: 70


> Train Language Model

In [7]:
!pip install --upgrade jax jaxlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 MB 19.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.23.dev20240116
    Uninstalling jaxlib-0.4.23.dev20240116:
      Successfully uninstalled jaxlib-0.4.23.dev20240116
  Attempting uninstall: jax
    Found existing installation: jax 0.4.23
    Uninstalling jax-0.4.23:
      Successfully uninstalled jax-0.4.23


In [8]:
import pandas as pd
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

2024-04-07 05:46:55.217350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 05:46:55.217445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 05:46:55.381857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
# Load the dataset from CSV file
file_path = '/kaggle/input/telegram-spam-or-ham/dataset.csv'
df = pd.read_csv(file_path)

In [10]:
# Filter spam and ham messages
spam_df = df[df['text_type'] == 'spam']
ham_df = df[df['text_type'] == 'ham']

In [11]:
# Combine spam and ham messages
df = pd.concat([spam_df, ham_df])

In [12]:
# Extract text and text type columns
lines = df['text'].tolist()

In [13]:
# Integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [14]:
# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [15]:
# Pad sequences to ensure consistent length
max_sequence_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='pre')

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset from CSV file
file_path = '/kaggle/input/telegram-spam-or-ham/dataset.csv'
df = pd.read_csv(file_path)

# Assuming 'data' is the DataFrame loaded from the CSV
le = LabelEncoder()
df['target'] = le.fit_transform(df['text'])

# Define problem
vocab_size = 10000  # Assuming 'word_index' is predefined, otherwise adjust this value
max_length = 100

# Convert text to sequences and pad sequences
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=8))  # Removed input_length parameter
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, df['target'], epochs=10, batch_size=32, validation_split=0.2)

# Summarize the model
print(model.summary())

Epoch 1/10
107/509 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: -109044.0703

I0000 00:00:1712468835.415304      95 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0000e+00 - loss: -8615766.0000 - val_accuracy: 9.8280e-04 - val_loss: -152776400.0000
Epoch 2/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -148941696.0000 - val_accuracy: 9.8280e-04 - val_loss: -614914368.0000
Epoch 3/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -461622528.0000 - val_accuracy: 9.8280e-04 - val_loss: -1333794048.0000
Epoch 4/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -926077120.0000 - val_accuracy: 9.8280e-04 - val_loss: -2270014720.0000
Epoch 5/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -1517213312.0000 - val_accuracy: 9.8280e-04 - val_loss: -3399120896.0000
Epoch 6/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: -2236388864.0000 - val_accuracy: 9.8280e-04 - val_loss: -4702202368.0000
Epoch 7/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 8)         │        80,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           801 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242,405 (946.90 KB)

 Trainable params: 80,801 (315.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 161,604 (631.27 KB)

None


> Use Language Model

In [35]:
import pandas as pd
from random import randint
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Load cleaned text sequences from CSV file
df = pd.read_csv('/kaggle/input/telegram-spam-or-ham/dataset.csv')
lines = df['text'].tolist()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
vocab_size = len(tokenizer.word_index) + 1

# Generate sequences
sequences = tokenizer.texts_to_sequences(lines)
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

# Prepare input and output sequences
X = sequences[:, :-1]
y = sequences[:, -1]

# Define the model
model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, verbose=2)

# Function to generate sequence
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat.argmax():
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

# Select a seed text
seed_text = lines[randint(0, len(lines))]
print(seed_text + '\n')

# Generate new text
generated = generate_seq(model, tokenizer, max_length-1, seed_text, 50)
print(generated)

Epoch 1/50
636/636 - 27s - 42ms/step - accuracy: 0.0344 - loss: 9.1768
Epoch 2/50
636/636 - 23s - 36ms/step - accuracy: 0.0412 - loss: 7.8149
Epoch 3/50
636/636 - 23s - 36ms/step - accuracy: 0.0508 - loss: 7.2957
Epoch 4/50
636/636 - 23s - 36ms/step - accuracy: 0.0647 - loss: 6.8838
Epoch 5/50
636/636 - 23s - 36ms/step - accuracy: 0.0788 - loss: 6.5042
Epoch 6/50
636/636 - 23s - 36ms/step - accuracy: 0.0977 - loss: 6.1140
Epoch 7/50
636/636 - 23s - 36ms/step - accuracy: 0.1177 - loss: 5.7538
Epoch 8/50
636/636 - 23s - 36ms/step - accuracy: 0.1374 - loss: 5.4063
Epoch 9/50
636/636 - 23s - 36ms/step - accuracy: 0.1618 - loss: 5.0757
Epoch 10/50
636/636 - 23s - 36ms/step - accuracy: 0.1852 - loss: 4.7564
Epoch 11/50
636/636 - 23s - 36ms/step - accuracy: 0.2107 - loss: 4.4469
Epoch 12/50
636/636 - 23s - 36ms/step - accuracy: 0.2446 - loss: 4.1403
Epoch 13/50
636/636 - 23s - 36ms/step - accuracy: 0.2831 - loss: 3.8458
Epoch 14/50
636/636 - 41s - 65ms/step - accuracy: 0.3299 - loss: 3.5514
E